# Request Type Analysis

Look at the request type values from 311.  Questions to consider:

  - Counts
  - Spatial (NCs) distribution
  - Time to complete
  - Time to complete by service provider
  - Spatial (service region) distr
  - Repeated addresses
  
**Note:** Compendium of hacks to examine/subset the gdf.  Explore!

# 1 - Setup

In [ ]:
%run start.py
from utils import read_new311_shape, dt_to_object

# 2 - Get Data Files

Two data sets:

  1. extended311 for point features
  2. cleaned, certified NCs for polygons

In [ ]:
%%time
#extended311_gdf = read_new311_shape('../data/311/extended311-geo-shape.shp')
extended311_gdf = gpd.read_parquet('../data/311/extended311-geo-shape.parq')

In [ ]:
extended311_gdf.info()

Certified, cleaned neighborhoods is a common idiom at this stage so ...

In [ ]:
neighborhoods_gdf = gpd.read_file('../data/neighborhoods/Neighborhood_Councils_(Certified)_cleaned.shp')

neighborhoods_gdf.rename(columns={'NAME': 'name',
                        'NC_ID': 'nc_id',
                        'SERVICE_RE': 'service_region'},
              inplace=True);

In [ ]:
neighborhoods_gdf.info()

# 3 - Some Data Massaging/Exploring

Various ideas to explore the gdf via request_type.  

Also would be good to look at the "repeat offenders" by looking at common addresses (note location by block face so ...)

In [ ]:
extended311_gdf.iloc[27]

In [ ]:
extended311_gdf.iloc[27]['created_dt'].day_of_week

In [ ]:
extended311_gdf.iloc[27]['created_dt'].date()

In [ ]:
extended311_gdf['day_of_week'] = extended311_gdf['created_dt'].apply(lambda dt: dt.day_of_week)

In [ ]:
extended311_gdf.day_of_week.value_counts()

In [ ]:
extended311_gdf['date'] = extended311_gdf['created_dt'].apply(lambda dt: dt.date())

In [ ]:
extended311_gdf['date'].value_counts(sort=False)

In [ ]:
extended311_gdf['month'] = extended311_gdf['created_dt'].apply(lambda dt: dt.month)

In [ ]:
extended311_gdf['month'].value_counts(sort=False)

In [ ]:
extended311_gdf['quarter'] = extended311_gdf['created_dt'].apply(lambda dt: dt.quarter)

In [ ]:
extended311_gdf['quarter'].value_counts(sort=False)

In [ ]:
still_open_gdf = extended311_gdf[extended311_gdf['closed_dt'].isnull()].reset_index()

In [ ]:
pd.options.display.max_rows

In [ ]:
pd.set_option("max_rows", 200)
pd.set_option("min_rows", 20)
still_open_gdf['date'].value_counts(sort=False, dropna=False).to_frame().reset_index()
#pd.reset_option("max_rows")

In [ ]:
extended311_gdf_info = Output(layout={'border': '1px solid black',
                            'width': '50%'})

still_open_gdf_info = Output(layout={'border': '1px solid black',
                            'width': '50%'})

with extended311_gdf_info:
    display(HTML('<center><b>created count</b></center>'))
    display(extended311_gdf['date'].value_counts(sort=False))

with still_open_gdf_info:
    display(HTML('<center><b>still open count</b></center>'))
    display(still_open_gdf['date'].value_counts(sort=False))

HBox([extended311_gdf_info, still_open_gdf_info])

In [ ]:
f1 = extended311_gdf['date'].value_counts(sort=False).to_frame().reset_index().rename(columns={'index': 'day', 'date': 'created count'})
f2 = still_open_gdf['date'].value_counts(sort=False).to_frame().reset_index().rename(columns={'index': 'day', 'date': 'open count'})   

merged_counts = pd.merge(f1, f2, on="day")
merged_counts['percentage'] = merged_counts.apply(lambda row: row['open count']/row['created count'], axis=1)

In [ ]:
merged_counts

In [ ]:
graffiti_gdf = gpd.read_parquet('../data/311/graffiti.parq')

In [ ]:
graffiti_counts = graffiti_gdf['nc'].value_counts().to_frame().reset_index().rename(columns={'index': 'nc_id', 'nc': 'count'})

In [ ]:
graffiti_counts.head()

In [ ]:
len(graffiti_gdf)

In [ ]:
graffiti_merged = pd.merge(neighborhoods_gdf, graffiti_counts, how="left", on=["nc_id"])

In [ ]:
graffiti_merged.head()

In [ ]:
graffiti_gdf['address'].value_counts()

In [ ]:
graffiti_gdf[graffiti_gdf['nc_name'].notnull()].query(f"nc_name.str.contains('South Central')", engine="python")['address'].value_counts()

# 4 - Display Counts

Display counts as choropleth using ipyleaflet.  

In [ ]:
from ipyleaflet import FullScreenControl

In [ ]:
imagery = basemap_to_tiles(basemaps.Esri.WorldImagery)
imagery.base = True
osm = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)
osm.base = True


map_display = Map(center=(34.05, -118.25), zoom=11,
                  layers=[imagery, osm],
                  layout=Layout(height="900px"),
                  scroll_wheel_zoom=True)

#map_display.add_control(LayersControl())
#map_display += nc_layer

map_display.add_control(FullScreenControl())
map_display

refer to : https://www.youtube.com/watch?v=wjzAy_yLrdA

In [ ]:
from ipyleaflet import Choropleth, Map
from branca.colormap import linear
a_geojson = json.loads(graffiti_merged.to_json())

graffiti_density = dict(zip(graffiti_merged['name'].tolist(), graffiti_merged['count'].tolist()))
for i in a_geojson['features']:
    i['id'] = i['properties']['name']

layer = Choropleth(
                    geo_data=a_geojson,
                    choro_data=graffiti_density,
                    colormap=linear.YlOrRd_09, #linear.Blues_05,
                    style={'fillOpacity': 1.0, "color":"black"},)
                    #key_on="name")

map_display.add_layer(layer)

I need to revisit a tooltip type popup.  For now this will work.

In [ ]:
geo_json = GeoJSON(
    data=a_geojson,
    style={
        'opacity': 1, 'dashArray': '9', 'fillOpacity': 0.6, 'weight': 1
    },
    hover_style={
        'color': 'white', 'dashArray': '0', 'fillOpacity': 0.5
    },
    name='NCs'
)

html = HTML('''Hover over a district''')
html.layout.margin = '0px 20px 20px 20 px'
control = WidgetControl(widget=html, position='bottomright')

def update_html(feature, **kwargs):
    html.value = '''<h3><b>NC: {}</b></h3>
                    <h4>Count: {}'''.format(feature['properties']['name'],
                                                           feature['properties']['count'])
    
map_display.add_control(control)  # does += work for this?

layer.on_hover(update_html)

# 6 - So What?

This is a basic viz using ipyleaflet.  What's next:

  1. I have two other visualization notebooks in this repo using folium.  Check them out.
  2. I would like to hook this up with bqplot.
  3. I've recently (01/12/2022) started evaluating holoviz...this should integrate with that.
  
